A mixture of $N =10$ random Gaussians centered in $[-1,1]^n$ 
$$
f_{\mu,\sigma}(x, y) = \sum_{i=1}^{N} \frac{1}{\sigma_i\sqrt{2\pi}}\exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$
 is defined over the square $[-1, 1]^2$.

In [1]:
using Pkg
Pkg.activate("../../../.")
using Globtim

# Constants and Parameters
d = 1 # Initial Degree 
const n, a, b = 3, 1, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 8 / 10  # Sampling parameters
const tol_l2 = 1.e-6             # Define the tolerance for the L2-norm

  Activating project at `~/Globtim.jl`


1.0e-6

In [20]:
N = 3
params = init_gaussian_params(n, N, 0.4)
# Create a closure that captures params
rand_gaussian_closure = (x) -> rand_gaussian(x, params)
f = rand_gaussian_closure; # Function to be optimized

The type of `poly_approx` is `ApproxPoly`.
It has fields: 
    `
    
    ApproxPoly::{T<:Number}
    coeffs::Vector{T} 
    degree::Int
    nrm::Float64
    N::Int
    scale_factor::Float64
    grid::Matrix{Float64}
    z::Vector{Float64}
    ` 

In [21]:
while true && d < 4# Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 1.) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;
println("Optimal degree: ", d)
println("Optimal L2-norm: ", poly_approx.nrm)
println("Optimal number of samples: ", poly_approx.N)
println("optimal polynomial: ",poly_approx.coeffs)
println("optimal polynomial: ",eltype(poly_approx.coeffs))

Optimal degree: 4
Optimal L2-norm: 0.0002613491715253416
Optimal number of samples: 21
optimal polynomial: [0.001678398186634644, -0.0002651968977090733, -0.002732237094956925, 0.0010187811942785811, -0.0011673279961386164, 0.0009497098147135598, 0.0025182791343234087, -0.0012199908699920188, 0.0005234444548009591, 0.0023393837838675277, 0.0012606189161794138, -0.00015861363943195053, -0.002019376584251966, -0.0008121422069703141, 0.0007497596984938905, -0.000888393415339814, -0.0023023987890348706, 0.0004130113428931972, 0.0016797133184820428, -0.0028318386518459667]
optimal polynomial: Float64


Now we run some analysis on the approximation of the function $f_{\mu,\sigma}$ by a polynomial of degree $d$. We want to measure the distribution of the error. Compare with a uniform grid and a Chebyshev grid. We shall run statistics on the error distribution.

In [30]:
using IterTools

function uniform_grid(n; range_min=-1.0, range_max=1.0, num_points_per_dim=10)
    # Create a range of points for each dimension
    ranges = [range(range_min, stop=range_max, length=num_points_per_dim) for _ in 1:n]

    # Generate the Cartesian product of the ranges to create the grid
    grid_points = collect(IterTools.product(ranges...))
    # Convert the grid points to a matrix where each row is a point
    uniform_grid_matrix = reduce(hcat, map(x -> collect(x), grid_points))'
    # Print the grid points
    uniform_grid_vectors = [collect(row) for row in eachrow(uniform_grid_matrix)]
    return uniform_grid_vectors
end

test_grid = uniform_grid(n)
f_values_uniform = f.(test_grid);

poly_approx.grid
chebyshev_grid_vectors = [collect(row) for row in eachrow(poly_approx.grid)]
f_values_chebyshev = f.(chebyshev_grid_vectors);


In [76]:
using GLMakie

# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
x_coords = coords[:, 1]
y_coords = coords[:, 2]
z_coords = coords[:, 3]

# Set a threshold for "close to zero" values
threshold = 0.1  # Adjust as necessary

# Filter data based on proximity to zero
mask = abs.(f_values_chebyshev) .> threshold
filtered_x = x_coords[mask]
filtered_y = y_coords[mask]
filtered_z = z_coords[mask]
filtered_f_values = f_values_chebyshev[mask]

# Normalize the filtered_f_values to the range [0, 1]
min_f_value = minimum(filtered_f_values)
max_f_value = maximum(filtered_f_values)
normalized_f_values = (filtered_f_values .- min_f_value) ./ (max_f_value - min_f_value)

# Plot the filtered points with varying transparency
fig = Figure()
ax = Axis3(fig[1, 1], title="Scatter Plot of Coordinates", xlabel="X", ylabel="Y", zlabel="Z")

scatter!(
    ax, filtered_x, filtered_y, filtered_z,
    markersize=5,
    color=normalized_f_values,
    colormap=:viridis)

# Display the plot
fig

In [77]:
# alpha_values = map(z -> abs(z) < threshold ? 0.2 : 0.0, filtered_f_values)

In [78]:
fig_combined = Figure(resolution = (1600, 400))
ax_combined_chebyshev = Axis(fig_combined[1, 1], title="Histogram of f_values (Chebyshev)", xlabel="f(x)", ylabel="Frequency")
ax_combined_uniform = Axis(fig_combined[1, 2], title="Histogram of f_values (Uniform)", xlabel="f(x)", ylabel="Frequency")

# Plot the histograms
hist!(ax_combined_chebyshev, f_values_chebyshev, bins=120, color=:blue)
hist!(ax_combined_uniform, f_values_uniform, bins=120, color=:red)

display(fig_combined)


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie /Users/ghscholt/.julia/packages/Makie/6c4lt/src/scenes.jl:229


GLMakie.Screen(...)

In [87]:
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

@polyvar(x[1:n]) 
"# Quick dirty fix the degree -1. But main_nd checks if the dimension is correct."
pol = main_nd(x, n, d-1, poly_approx.coeffs) # Quick dirty fix the degree -1. 

pol

We now check the evaluations of the polynomial, in order to then plot the distribution of the errors too. 

In [86]:
result_cheb = [subs(pol, x[1] => p[1], x[2] => p[2], x[3] => p[3]) for p in chebyshev_grid_vectors]
result_cheb_mask = result_cheb[mask]

1069-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, BigFloat}}:
 0.003392780074047647783226231134638732523451054879502789983196403659027782183637417
 0.002946932135646556584012790016804021152457390084627577956678612656021082157587984
 0.003029344468199942273735501224779692327880081181273125586401766098904771704926342
 0.004367358074624776316952839313970502696300289672406444193040090445080799061328028
 0.004465443809318496543870315552463134886737340383227610644200353233828475150719688
 0.004150873615919429251602999862633397600350269232381481447980539225529744571307686
 0.005901315093755349897785194907204816736523511794692122364871757228714991184625115
 0.006009762397845448979604980507595686503289472174551632945845496678740072220726437
 0.00316500891654540808597758304759552457989825360720620422527593987139494675626776
 0.003826442482232979555874692357575346436498132535330008713284827149614991909761452
 ⋮
 0.002820643695226683

In [ ]:
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d - 1)

In [7]:
# using WGLMakie

# fig = Figure()
# ax = Axis3(fig[1, 1], title="Function f on Uniform Grid", xlabel="X", ylabel="Y", zlabel="f(x)")

# # Extract the x, y, and z coordinates from the grid
# x_coords = test_grid[1, :]
# y_coords = test_grid[2, :]
# z_coords = test_grid[3, :]

# # Plot the points with the function values as the color
# scatter!(ax, x_coords, y_coords, z_coords, markersize=5, color=f_values, colormap=:viridis)

# # Display the plot
# fig

In [8]:
# Real_sol_lstsq = HomotopyContinuation.solve(sys)
# real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

In [9]:
# function condition(point::Vector{Float64}, n::Int)::Bool
#     return all(-1 < point[i] < 1 for i in 1:n)
# end
# # condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
# # filtered_points = filter(condition, real_pts) # Filter points using the filter function
# filtered_points = filter(p -> condition(p, n), real_pts) # Filter points using the condition function

# # Colllect the critical points of the approximant 
# # h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
# # h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
# # h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
# # df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z) # Create a DataFrame

Repeat solving with exact method, compare timing. 

In [11]:
# loc = "inputs.ms"
# # File path of the output file
# file_path_output = "outputs.ms";

# using DynamicPolynomials, DataFrames
# ap = main_nd(n, d, poly_approx.coeffs)
# @polyvar(x[1:n]) # Define polynomial ring 
# # Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x
# names = [x[i].name for i in 1:length(x)]
# open(loc, "w") do file
#     println(file, join(names, ", "))
#     println(file, 0)
# end
# # Define the polynomial approximant 
# PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
# for i in 1:n
#     partial = differentiate(PolynomialApproximant, x[i])
#     partial_str = replace(string(partial), "//" => "/")
#     open(loc, "a") do file
#         if i < n
#             println(file, string(partial_str, ","))
#         else
#             println(file, partial_str)
#         end
#     end
# end

# # Optimize the collected entries 
# using Optim
# for i in 1:nrow(df)
#     println("Optimizing for point $i")
#     x0 = [df.x[i], df.y[i]]
#     res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
#     minimizer = Optim.minimizer(res)
#     min_value = Optim.minimum(res)
#     steps = res.iterations
#     converged = Optim.converged(res)
#     distance = norm(x0 - minimizer)
#     println(summary(res))
# end


Should plot the polynomial approximant too.